### MSA 8040 Final Project
### Team: Paul Bawool Kim, Deepa Naishadham, Gowri Jaganathan Sengundermudaliar

### BeautifulSoup Library to extract Hotel Specification data

In [1]:
import requests
## right click on bubbles
r1=requests.get('https://www.tripadvisor.com/Hotels-g60898-Atlanta_Georgia-Hotels.html')
r2=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa30-Atlanta_Georgia-Hotels.html')
r3=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa60-Atlanta_Georgia-Hotels.html')
r4=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa90-Atlanta_Georgia-Hotels.html')
r5=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa120-Atlanta_Georgia-Hotels.html')
r6=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa150-Atlanta_Georgia-Hotels.html')
r7=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa180-Atlanta_Georgia-Hotels.html')
r8=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa210-Atlanta_Georgia-Hotels.html')
r9=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa240-Atlanta_Georgia-Hotels.html')
r10=requests.get('https://www.tripadvisor.com/Hotels-g60898-oa270-Atlanta_Georgia-Hotels.html')


In [2]:
print(r9.text[0:100])

<!DOCTYPE html><html lang="en"><head><meta http-equiv="content-type" content="text/html; charset=utf


#### Parsing the HTML using Beautiful Soup

In [3]:
from bs4 import BeautifulSoup

##  The code below parses the HTML (stored in r.text) into a special object called soup 
##  that the Beautiful Soup library understands. 
##  Beautiful Soup reads the HTML and makes sense of its structure

soup1=BeautifulSoup(r1.text, 'html.parser')
soup2=BeautifulSoup(r2.text, 'html.parser')
soup3=BeautifulSoup(r3.text, 'html.parser')
soup4=BeautifulSoup(r4.text, 'html.parser')
soup5=BeautifulSoup(r5.text, 'html.parser')
soup6=BeautifulSoup(r6.text, 'html.parser')
soup7=BeautifulSoup(r7.text, 'html.parser')
soup8=BeautifulSoup(r8.text, 'html.parser')
soup9=BeautifulSoup(r9.text, 'html.parser')
soup10=BeautifulSoup(r10.text, 'html.parser')

In [4]:

## Now, we request Beautiful Soup to find all of the records
#results=soup1.find_all('div', attrs={'class':'prw_rup'})

results=soup1.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '})
results.extend(soup2.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup3.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup4.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup5.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup6.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup7.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup8.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup9.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))
results.extend(soup10.find_all('div', attrs={'class':'ui_column is-8 main_col allowEllipsis '}))



In [5]:
## results acts like a Python list, so we can check its length
len(results)

290

### Building the dataset

In [6]:
records= []
for result in results:
    
    hotelname_elt=result.find('a', attrs={"data-clicksource":"HotelName"})
    hotel_name=hotelname_elt.text
    
    hotelurl=result.find('a', attrs={'target':'_blank'})
    hotel_URL=hotelurl['href']
    
    bubble_elt=result.find('span', attrs={'class':'ui_bubble_rating'})
    # Some hotels have no reviews
    if bubble_elt is not None:
        bubble_rating1=bubble_elt['alt']
        bubble_rating=float(bubble_rating1.split(" ")[0])
    else:
        bubble_rating=None
        
    review_elt=result.find('a', attrs={'class':'review_count'})
    Number_of_reviews1=review_elt.text
    Number_of_reviews2=Number_of_reviews1.split(" ")[0]           ## Saving the numeric value from (say) "3,218 reviews"
    Number_of_reviews=int(Number_of_reviews2.replace(',', ''))    ## to convert a string output from prev row to an integer need to make 3,218 to 3218
    Number_of_reviews
          
    price_elt=result.find('div', attrs={'class':'price-wrap '})
    # Some hotels require you to click to view prices (they are not displayed on webpage)
    if price_elt is not None and price_elt.text != '':
        #print(price_elt.text)
        hotel_price1=price_elt.text
        hotel_price=int(hotel_price1.split("$")[1])
    else:
        hotel_price = None
    
    amenities_elts=result.find_all('div', attrs={'class':'label'})  
    len(amenities_elts)
    
    amenities_list=[]           # initialize an empty list to fill
    for amenity_elt in amenities_elts:
        amenities_list.append(amenity_elt.text)
    
    records.append((hotel_name,hotel_URL,bubble_rating,Number_of_reviews,hotel_price,amenities_list))
records 

print(records[2])

('Holiday Inn & Suites Atlanta Airport - North', '/Hotel_Review-g60898-d86197-Reviews-Holiday_Inn_Suites_Atlanta_Airport_North-Atlanta_Georgia.html', 4.5, 837, 100, ['Free Wifi', 'Pool', 'Visit hotel website'])


In [7]:
import pandas as pd
## Create a list of all elements without the amenities to include in a dataframe
list1=[]
for record in records:
    list1.append(record[:-1])
#print(list1)
## To create a dataframe from this list.
labels = ['HotelName', 'HotelURL', 'BubbleRating out of 5', 'ReviewCount', 'Price$']
df_hotels = pd.DataFrame.from_records(list1, columns=labels)    ##from_record is a function in the dataframe class
df_hotels
df_hotels.to_csv("Hotels_02142018.csv", index=True, header=True)

### To get a list of all amenities

In [8]:
# Create a set of all amenities to add to the dataframe
All_amenities=set()

for record in records:
    for i in range(len(record[5])):
        All_amenities.add(record[5][i])
print(All_amenities)

{'Room Service', 'Free Parking', 'Restaurant', 'Breakfast included', 'Pool', 'Visit hotel website', 'Free Wifi', 'Bar/Lounge', 'Fitness center', 'Free Internet'}


In [9]:
df_amenities=pd.DataFrame(columns=All_amenities)   # since list of amenities can change we will call in the list and not hand type list 
df_amenities
dfhotel2 = pd.concat([df_hotels, df_amenities], axis=1, sort=False)
dfhotel2

,HotelName,HotelURL,BubbleRating out of 5,ReviewCount,Price$,Room Service,Free Parking,Restaurant,Breakfast included,Pool,Visit hotel website,Free Wifi,Bar/Lounge,Fitness center,Free Internet
0,Holiday Inn Atlanta - Northlake,/Hotel_Review-g60898-d223222-Reviews-Holiday_I...,3.5,165,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"The Westin Peachtree Plaza, Atlanta",/Hotel_Review-g60898-d114387-Reviews-The_Westi...,4.0,3235,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Holiday Inn & Suites Atlanta Airport - North,/Hotel_Review-g60898-d86197-Reviews-Holiday_In...,4.5,837,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grand Hyatt Atlanta in Buckhead,/Hotel_Review-g60898-d86179-Reviews-Grand_Hyat...,4.5,2325,209.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wingate by Wyndham Atlanta/Buckhead,/Hotel_Review-g60898-d223001-Reviews-Wingate_b...,4.5,2475,109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,La Quinta Inn & Suites Atlanta Airport North,/Hotel_Review-g60898-d224838-Reviews-La_Quinta...,4.5,465,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,InterContinental Buckhead Atlanta,/Hotel_Review-g60898-d513075-Reviews-InterCont...,4.5,3074,158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,The Georgian Terrace Hotel,/Hotel_Review-g60898-d111371-Reviews-The_Georg...,4.0,2023,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Emory Conference Center Hotel,/Hotel_Review-g60898-d181714-Reviews-Emory_Con...,4.5,1391,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Georgia Tech Hotel and Conference Center,/Hotel_Review-g60898-d281240-Reviews-Georgia_T...,4.5,1141,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html
# original = {"box": 1, "cat": 2, "apple": 5}
# original["cat"] = 200                        ## change values in a dictionary

# values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}   dictionary with key and values
# df.fillna(value=values)

amenities_list={}       # create a blank dictionary
for amenity in All_amenities:
    amenities_list[amenity]=0  ## 'No'
    
dfhotel2=dfhotel2.fillna(value=amenities_list)
#amenities_list 
dfhotel2


,HotelName,HotelURL,BubbleRating out of 5,ReviewCount,Price$,Room Service,Free Parking,Restaurant,Breakfast included,Pool,Visit hotel website,Free Wifi,Bar/Lounge,Fitness center,Free Internet
0,Holiday Inn Atlanta - Northlake,/Hotel_Review-g60898-d223222-Reviews-Holiday_I...,3.5,165,89.0,0,0,0,0,0,0,0,0,0,0
1,"The Westin Peachtree Plaza, Atlanta",/Hotel_Review-g60898-d114387-Reviews-The_Westi...,4.0,3235,101.0,0,0,0,0,0,0,0,0,0,0
2,Holiday Inn & Suites Atlanta Airport - North,/Hotel_Review-g60898-d86197-Reviews-Holiday_In...,4.5,837,100.0,0,0,0,0,0,0,0,0,0,0
3,Grand Hyatt Atlanta in Buckhead,/Hotel_Review-g60898-d86179-Reviews-Grand_Hyat...,4.5,2325,209.0,0,0,0,0,0,0,0,0,0,0
4,Wingate by Wyndham Atlanta/Buckhead,/Hotel_Review-g60898-d223001-Reviews-Wingate_b...,4.5,2475,109.0,0,0,0,0,0,0,0,0,0,0
5,La Quinta Inn & Suites Atlanta Airport North,/Hotel_Review-g60898-d224838-Reviews-La_Quinta...,4.5,465,140.0,0,0,0,0,0,0,0,0,0,0
6,InterContinental Buckhead Atlanta,/Hotel_Review-g60898-d513075-Reviews-InterCont...,4.5,3074,158.0,0,0,0,0,0,0,0,0,0,0
7,The Georgian Terrace Hotel,/Hotel_Review-g60898-d111371-Reviews-The_Georg...,4.0,2023,208.0,0,0,0,0,0,0,0,0,0,0
8,Emory Conference Center Hotel,/Hotel_Review-g60898-d181714-Reviews-Emory_Con...,4.5,1391,107.0,0,0,0,0,0,0,0,0,0,0
9,Georgia Tech Hotel and Conference Center,/Hotel_Review-g60898-d281240-Reviews-Georgia_T...,4.5,1141,116.0,0,0,0,0,0,0,0,0,0,0


In [11]:
##  We have 2 data locations (records(list) and dfhotel2(dataframe).
##  First we need to connect these with a common identifier--here we use hotel name

for record in records:
    HotelName=record[0]   #  we pull the hotel name from records
#    print(HotelName,record[5])     
## Now we will find the row(location) in dfhotel2 where the same hotel is located
    dfhotel2.loc[dfhotel2['HotelName'] == HotelName]  # inner block is Boolean (checks if hotel name in dfhotel2 matched hotelname in records list)
                                                      #  loc uses name or Boolean instead of index of a row element
## We now loop thru the amenities for the current record and set that row/column for that Hotel/amenity to 1
#    print(record[-1])
    for amenity in record[-1]:
#        print(amenity)
        dfhotel2.loc[dfhotel2['HotelName'] == HotelName,amenity]=1  # 'Yes'

# Since the dfhotel2 dataframe includes sponsored hotels there are duplicate hotels. Need to remove these duplicates
dfhotel2=dfhotel2.drop_duplicates()
   
dfhotel2.to_csv("dfHotels_02142018.csv", index=True, header=True, sep=',') 

In [12]:
dfhotel2

,HotelName,HotelURL,BubbleRating out of 5,ReviewCount,Price$,Room Service,Free Parking,Restaurant,Breakfast included,Pool,Visit hotel website,Free Wifi,Bar/Lounge,Fitness center,Free Internet
0,Holiday Inn Atlanta - Northlake,/Hotel_Review-g60898-d223222-Reviews-Holiday_I...,3.5,165,89.0,0,1,0,0,0,1,1,0,0,0
1,"The Westin Peachtree Plaza, Atlanta",/Hotel_Review-g60898-d114387-Reviews-The_Westi...,4.0,3235,101.0,0,0,1,0,1,1,0,0,0,0
2,Holiday Inn & Suites Atlanta Airport - North,/Hotel_Review-g60898-d86197-Reviews-Holiday_In...,4.5,837,100.0,0,0,0,0,1,1,1,0,0,0
3,Grand Hyatt Atlanta in Buckhead,/Hotel_Review-g60898-d86179-Reviews-Grand_Hyat...,4.5,2325,209.0,0,0,0,0,1,1,1,0,0,0
4,Wingate by Wyndham Atlanta/Buckhead,/Hotel_Review-g60898-d223001-Reviews-Wingate_b...,4.5,2475,109.0,0,1,0,0,0,0,1,0,0,0
5,La Quinta Inn & Suites Atlanta Airport North,/Hotel_Review-g60898-d224838-Reviews-La_Quinta...,4.5,465,140.0,0,1,0,0,0,1,1,0,0,0
6,InterContinental Buckhead Atlanta,/Hotel_Review-g60898-d513075-Reviews-InterCont...,4.5,3074,158.0,0,0,1,0,1,1,0,0,0,0
7,The Georgian Terrace Hotel,/Hotel_Review-g60898-d111371-Reviews-The_Georg...,4.0,2023,208.0,0,0,0,0,1,1,1,0,0,0
8,Emory Conference Center Hotel,/Hotel_Review-g60898-d181714-Reviews-Emory_Con...,4.5,1391,107.0,0,0,0,0,1,0,1,0,0,0
9,Georgia Tech Hotel and Conference Center,/Hotel_Review-g60898-d281240-Reviews-Georgia_T...,4.5,1141,116.0,0,0,0,0,1,0,1,0,0,0
